In [1]:
from sklearn.model_selection import cross_validate
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

In [2]:
%matplotlib inline 

from IPython.display import Image
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import seaborn as sns

In [3]:
# ignorando alertas (não erros)

import warnings
warnings.filterwarnings('ignore')

In [4]:
# importando o dataset
df = pd.read_csv('dataset/data-set-clube-new.csv', sep=';')